In [1]:
import os
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.48


In [2]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

# Compute Environment

In [3]:
cluster = 'gandalf'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC12', min_nodes=1, max_nodes=5)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)
    

Found existing compute target


# Run Experiment

In [4]:
# Create and run experiment
mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

run = mnist.submit(estimator)

In [5]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
mnist_lab,mnist_lab_1563571422_1b966819,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [6]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Exception in thread Thread-20:
Traceback (most recent call last):
  File "c:\programdata\miniconda3\envs\tf\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "c:\programdata\miniconda3\envs\tf\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "c:\programdata\miniconda3\envs\tf\lib\multiprocessing\pool.py", line 486, in _handle_results
    cache[job]._set(i, obj)
  File "c:\programdata\miniconda3\envs\tf\lib\multiprocessing\pool.py", line 662, in _set
    self._callback(self._value)
  File "c:\programdata\miniconda3\envs\tf\lib\site-packages\azureml\widgets\_userrun\_run_details.py", line 503, in _update_metrics
    self.widget_instance.run_metrics = result
  File "c:\programdata\miniconda3\envs\tf\lib\site-packages\traitlets\traitlets.py", line 585, in __set__
    self.set(obj, value)
  File "c:\programdata\miniconda3\envs\tf\lib\site-packages\traitlets\traitlets.py", line 574, in set
    obj._notify_trait(self.name, old_value

# Manage Data

In [7]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = 'data', target_path = 'mnist', show_progress = True)

Exception: src_path must be a directory

In [ ]:
# run the same was as above
script_params={
    '--data': mnist_data.as_mount(),
    '
}

# Create and run experiment
mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

run = mnist.submit(estimator)
RunDetails(run).show()